# Cluster bulk list of keywords with valueserp


Quickly cluster a large quantity of keywords using SERP results

In [378]:
import pandas as pd
import requests
from io import BytesIO
from zipfile import ZipFile
import requests
import json

In [388]:
# inputs

api_key = "0E11249D01184FABAA637349023CCF79"

common_urls = 2

In [383]:

# load the keywords

filename = "organic-list-from-ahrefs.csv"
df_keywords_input = pd.read_csv(filename)
df_keywords_input.head()



,Keyword,SERP features,Volume,KD,CPC,Previous traffic,Current traffic,Traffic change,Previous position,Current position,Position change,Previous URL inside,Previous URL,Current URL inside,Current URL,Previous date,Current date
0,league of american bicyclists,"Sitelinks, Tweet box, Knowledge panel, People ...",1000,9,1.73,832.0,832.0,0.0,1.0,1.0,NaN,NaN,https://www.bikeleague.org/,NaN,https://www.bikeleague.org/,2022-11-14 18:26:04,2023-02-09 11:02:10
1,rules of the road bicycles,"Sitelinks, People also ask, Image pack, Videos",600,24,NaN,65.0,182.0,117.0,3.0,1.0,2,NaN,https://www.bikeleague.org/StateBikeLaws,NaN,https://www.bikeleague.org/content/rules-road-0,2022-11-12 20:31:42,2023-02-13 09:06:52
2,bike classes near me,"Local pack, Sitelinks, People also ask",500,26,1.61,73.0,180.0,108.0,2.0,1.0,1,NaN,https://www.bikeleague.org/content/find-take-c...,NaN,https://www.bikeleague.org/content/find-take-c...,2022-10-24 02:39:16,2023-02-03 20:54:25
3,bike to work day 2022,NaN,1600,46,NaN,146.0,NaN,NaN,4.0,NaN,Lost,NaN,https://www.bikeleague.org/content/bike-month-...,NaN,NaN,2022-11-11 20:23:28,NaN
4,rules for bicycles on the road,NaN,600,31,NaN,138.0,NaN,NaN,1.0,NaN,Lost,NaN,https://www.bikeleague.org/StateBikeLaws,NaN,NaN,2022-11-14 07:19:39,NaN


In [379]:
# create batch upload file
# possibly query serp results including the fist 30 not just first page


body = {
  "name": "My First Auto Batch",
  "enabled": True,
  "schedule_type": "manual",
  "priority": "normal"
}

api_result = requests.post('https://api.valueserp.com/batches?api_key='+api_key, json=body)

api_response = api_result.json()

print("Batch Created: ", json.dumps(api_response))


Batch Created:  {"request_info": {"success": true}, "batch": {"id": "D5A4837D", "created_at": "2023-02-14T14:51:12.718Z", "last_run": null, "name": "My First Auto Batch", "schedule_type": "manual", "enabled": true, "status": "idle", "searches_total_count": 0, "searches_page_count": 0, "credits_required": 0, "next_result_set_id": 1, "results_count": 0, "priority": "normal", "destination_ids": [], "searches_type": "mixed", "searches_type_locked": false}}


In [381]:
# upload batch file to valueserp and get batch_id

batch_id = api_response['batch']['id']
batch_id

'D5A4837D'

In [394]:
# upload all keywords as 'searches' in the batch

n_max = 1000

list_of_searches = []
list_of_search_to_batch_api_responses = []

for i, row in df_keywords_input.iterrows():
    
    search_for = {
        "q": row['Keyword'],
        "gl":"us",
        "hl":"en",
        "engine":"google",
        "location": "United States",
        "google_domain":"google.com",
        "device":"mobile",
        # "search_type": "web", # defaults to web
        # "custom_id": "MyCustomID_001"
    }
    
    list_of_searches.append(search_for)
    
    if len(list_of_searches) == 1000:
        print("sending batch", len(list_of_search_to_batch_api_responses))
        
        # send the max at a time
        body = {"searches": list_of_searches}

        api_result = requests.put('https://api.valueserp.com/batches/'+batch_id+'?api_key='+api_key, json=body)
        search_to_batch_api_response = api_result.json()
        list_of_search_to_batch_api_responses.append(search_to_batch_api_response)
        
        list_of_searches = []
    

# send the remaining searches to the batch
print("sending final batch", len(list_of_search_to_batch_api_responses))
body = {"searches": list_of_searches}

api_result = requests.put('https://api.valueserp.com/batches/'+batch_id+'?api_key='+api_key, json=body)
search_to_batch_api_response = api_result.json()
list_of_search_to_batch_api_responses.append(search_to_batch_api_response)

list_of_searches = []



sending batch 0
sending batch 1
sending batch 2
sending batch 3
sending batch 4
sending batch 5
sending batch 6
sending batch 7
sending final batch 8


In [396]:
list_of_search_to_batch_api_responses


[{'request_info': {'success': True},
  'batch': {'id': 'D5A4837D',
   'created_at': '2023-02-14T14:51:12.718Z',
   'last_run': None,
   'name': 'My First Auto Batch',
   'schedule_type': 'manual',
   'enabled': True,
   'status': 'idle',
   'api_requests_required': 1000,
   'searches_total_count': 1000,
   'searches_page_count': 1,
   'credits_required': 1000,
   'next_result_set_id': 1,
   'results_count': 0,
   'priority': 'normal',
   'destination_ids': [],
   'searches_type': 'web',
   'searches_per_type_count': {'web': 1000},
   'searches_type_locked': False}},
 {'request_info': {'success': True},
  'batch': {'id': 'D5A4837D',
   'created_at': '2023-02-14T14:51:12.718Z',
   'last_run': None,
   'name': 'My First Auto Batch',
   'schedule_type': 'manual',
   'enabled': True,
   'status': 'idle',
   'api_requests_required': 2000,
   'searches_total_count': 2000,
   'searches_page_count': 2,
   'credits_required': 2000,
   'next_result_set_id': 1,
   'results_count': 0,
   'priority'

In [398]:
# start the batch job

import requests

params = {
  'api_key': api_key
}

batch_api_result = requests.get('https://api.valueserp.com/batches/'+batch_id+'/start', params)

batch_api_response = batch_api_result.json()

print("Batch started.")
batch_api_response

Batch started.


{'request_info': {'success': True},
 'batch': {'id': 'D5A4837D',
  'created_at': '2023-02-14T14:51:12.718Z',
  'last_run': None,
  'name': 'My First Auto Batch',
  'schedule_type': 'manual',
  'enabled': True,
  'status': 'queued',
  'api_requests_required': 8047,
  'searches_total_count': 8047,
  'searches_page_count': 9,
  'credits_required': 8047,
  'next_result_set_id': 1,
  'results_count': 0,
  'priority': 'normal',
  'destination_ids': [],
  'searches_type': 'web',
  'searches_per_type_count': {'web': 8047},
  'searches_type_locked': False}}

In [404]:
# check for job completed


import requests

params = {
  'api_key': api_key
}

api_result = requests.get('https://api.valueserp.com/batches/'+batch_id, params)

api_response = api_result.json()

print("Batch Name: ", api_response['batch']['name'])

api_response

Batch Name:  My First Auto Batch


{'request_info': {'success': True},
 'batch': {'id': 'D5A4837D',
  'created_at': '2023-02-14T14:51:12.718Z',
  'last_run': '2023-02-14T15:16:00.123Z',
  'name': 'My First Auto Batch',
  'schedule_type': 'manual',
  'enabled': True,
  'status': 'idle',
  'api_requests_required': 8047,
  'searches_total_count': 8047,
  'searches_page_count': 9,
  'credits_required': 8047,
  'next_result_set_id': 2,
  'results_count': 1,
  'priority': 'normal',
  'destination_ids': [],
  'searches_type': 'web',
  'searches_per_type_count': {'web': 8047},
  'searches_type_locked': False}}

In [405]:
# get the link for the download

params = {
  'api_key': api_key
}

batch_resultset_api_result = requests.get('https://api.valueserp.com/batches/'+batch_id+'/results/1/csv', params)

batch_resultset_api_response = batch_resultset_api_result.json()
batch_resultset_api_response

{'request_info': {'success': True},
 'batch_id': 'D5A4837D',
 'result': {'id': 1,
  'started_at': '2023-02-14T15:16:00.123Z',
  'ended_at': '2023-02-14T15:19:25.799Z',
  'expires_at': '2023-02-28T15:16:00.123Z',
  'results_page_count': 9,
  'searches_completed': 8047,
  'searches_failed': 0,
  'searches_total': 8047,
  'webhook_status': {'status': 'not_applicable', 'log': []},
  'destination_status': {},
  'download_links': {'pages': ['https://data.valueserp.com/results/14_FEBRUARY_2023/1516/Batch_Results_D5A4837D_1_Page_1_002c0139d00c9dd8e4d1b13c504517942bd20489.csv',
    'https://data.valueserp.com/results/14_FEBRUARY_2023/1516/Batch_Results_D5A4837D_1_Page_2_002c0139d00c9dd8e4d1b13c504517942bd20489.csv',
    'https://data.valueserp.com/results/14_FEBRUARY_2023/1516/Batch_Results_D5A4837D_1_Page_3_002c0139d00c9dd8e4d1b13c504517942bd20489.csv',
    'https://data.valueserp.com/results/14_FEBRUARY_2023/1516/Batch_Results_D5A4837D_1_Page_4_002c0139d00c9dd8e4d1b13c504517942bd20489.csv',
 

In [412]:
zip_url = batch_resultset_api_response['result']['download_links']['all_pages']
zip_url

'https://data.valueserp.com/results/14_FEBRUARY_2023/1516/Batch_Results_D5A4837D_1_All_Pages_002c0139d00c9dd8e4d1b13c504517942bd20489_csv.zip'

In [413]:
# download zip file and open all the csvs inside

content = requests.get(zip_url)
zf = ZipFile(BytesIO(content.content))

File in zip: Batch_Results_D5A4837D_1_Page_1_002c0139d00c9dd8e4d1b13c504517942bd20489.csv
File in zip: Batch_Results_D5A4837D_1_Page_2_002c0139d00c9dd8e4d1b13c504517942bd20489.csv
File in zip: Batch_Results_D5A4837D_1_Page_3_002c0139d00c9dd8e4d1b13c504517942bd20489.csv
File in zip: Batch_Results_D5A4837D_1_Page_4_002c0139d00c9dd8e4d1b13c504517942bd20489.csv
File in zip: Batch_Results_D5A4837D_1_Page_5_002c0139d00c9dd8e4d1b13c504517942bd20489.csv
File in zip: Batch_Results_D5A4837D_1_Page_6_002c0139d00c9dd8e4d1b13c504517942bd20489.csv
File in zip: Batch_Results_D5A4837D_1_Page_7_002c0139d00c9dd8e4d1b13c504517942bd20489.csv
File in zip: Batch_Results_D5A4837D_1_Page_8_002c0139d00c9dd8e4d1b13c504517942bd20489.csv
File in zip: Batch_Results_D5A4837D_1_Page_9_002c0139d00c9dd8e4d1b13c504517942bd20489.csv


In [415]:
# find the csv files in the zip:

list_of_df_batch_pages = []

for csv_filename in [s for s in zf.namelist() if ".csv" in s]:
    print(csv_filename)

    df_batch_page = pd.read_csv(zf.open(csv_filename), low_memory=False)
    list_of_df_batch_pages.append(df_batch_page)
    

df = pd.concat(list_of_df_batch_pages)

Batch_Results_D5A4837D_1_Page_1_002c0139d00c9dd8e4d1b13c504517942bd20489.csv
Batch_Results_D5A4837D_1_Page_2_002c0139d00c9dd8e4d1b13c504517942bd20489.csv
Batch_Results_D5A4837D_1_Page_3_002c0139d00c9dd8e4d1b13c504517942bd20489.csv
Batch_Results_D5A4837D_1_Page_4_002c0139d00c9dd8e4d1b13c504517942bd20489.csv
Batch_Results_D5A4837D_1_Page_5_002c0139d00c9dd8e4d1b13c504517942bd20489.csv
Batch_Results_D5A4837D_1_Page_6_002c0139d00c9dd8e4d1b13c504517942bd20489.csv
Batch_Results_D5A4837D_1_Page_7_002c0139d00c9dd8e4d1b13c504517942bd20489.csv
Batch_Results_D5A4837D_1_Page_8_002c0139d00c9dd8e4d1b13c504517942bd20489.csv
Batch_Results_D5A4837D_1_Page_9_002c0139d00c9dd8e4d1b13c504517942bd20489.csv


In [418]:
# save the cluster file


delim = ";;"

# make dataframe from the serp lists
df = df[['result.organic_results.link', 'search.q', 'result.organic_results.position']]
df = df.rename(columns={'result.organic_results.link':'link', 'search.q':'query', 'result.organic_results.position':'position'})

# Clean input queries
df['query'] = df['query'].str.strip()

# distinct queries
queries = set(list(df['query']))
len(queries)

8047

In [419]:


# make position dataframe
df_position = df[["position", "query", "link"]].copy()

df = df.sort_values(by="query", ascending=True)
df = df.groupby('link')['query'].apply(delim.join).reset_index()
df["temp_count"] = (df["query"].str.count( delim ) + 1)
df = df[df["temp_count"] >= common_urls]

# keep only the longest version of each keyword group
list1 = df['query']
substrings = {w1 for w1 in list1 for w2 in list1 if w1 in w2 and w1 != w2}
longest_str = set(list1) - substrings
longest_str = list(longest_str)
df = df[df['query'].isin(longest_str)]

# drop any cluster groups under the minimum threshold
df['temp_count'] = df['query'].map(df.groupby('query')['query'].count())
df = df[df["temp_count"] >= common_urls]

# make the cluster column & delete helper column
df['serp_cluster'] = df['query']

# explode the keyword list
df['query'] = df['query'].str.split( delim )
df = df.explode('query')

# rename the cluster name to the shortest keyword
df['temp_count'] = df['query'].astype(str).map(len)
df = df.sort_values(by="temp_count", ascending=True)
df['serp_cluster'] = df.groupby('serp_cluster')['query'].transform('first')

del df['temp_count']
del df['link']

df.drop_duplicates(subset="query", inplace=True)

# add in no cluster results
df_no_cluster = pd.DataFrame(None)
queries_de_duped = list(set(queries))
df_no_cluster['query'] = queries_de_duped
df_no_cluster['serp_cluster'] = "zzz_no_cluster"
df_cluster = pd.concat([df_no_cluster, df])
df_cluster = df_cluster[['serp_cluster', 'query']]
df_cluster = df_cluster.sort_values(by="serp_cluster", ascending=True)
df_cluster.drop_duplicates(subset=['query'], keep="first", inplace=True)


df_cluster.groupby(['serp_cluster','query']).first()#.reset_index().groupby(['serp_cluster','query','temp_count']).count()

Empty DataFrame
Columns: []
Index: [((5, (5), ((5, >5), ((5, @5), (11 mile bike ride, 11 mile bike ride), (11 mile bike ride, biking 12 miles a day), (12 person bike, 12 person bike), (12 person bike, pedal bike indianapolis), (120 lakehurst dr bratenahl oh, 120 lakehurst dr bratenahl oh), (120 lakehurst dr bratenahl oh, 120 lakehurst dr. in bratenahl), (123 people search, 123 people search), (123 search, 123 search), (123 search, search 123), (12312 port grace blvd, 12312 port grace blvd), (12312 port grace blvd, 12312 port grace blvd la vista), (12312 port grace blvd, 12312 port grace blvd la vista ne 68128), (16 cars bike, 16 cars bicycle), (16 cars bike, 16 cars bike), (16516 s westland drive gaithersburg md 20877, 16516 s westland drive gaithersburg md 20877), (16516 s westland drive gaithersburg md 20877, 16516 s. westland drive gaithersburg, md 20877), (1716 locust street des moines, 1716 locust street des moines), (1716 locust street des moines, 1716 locust street, des moines, ia 50309), (17k steps in miles, 17k steps in miles), (17k steps in miles, 22k steps in miles), (1828 maple way coggon ia, 1828 maple way coggon ia), (1828 maple way coggon ia, 1828 maple way coggon iowa), (2 miles on a bike, 2 miles on a bike), (2 miles on a bike, how long to bike 2 miles), (2 miles on a bike, how long would it take to bike 2 miles), (2017 commuter bikes, 2017 commuter bikes), (2017 commuter bikes, commuter bikes 2017), (2021 bike, 2021 bike), (2021 bike, bicycling bike awards 2021), (2021 bike, bike of the year 2021), (21 baneberry dr cranston ri, 21 baneberry dr cranston ri), (21 baneberry dr cranston ri, 21 baneberry drive cranston ri), (22 bicycle, 22 bicycle), (22 bicycle, 22" bicycle), (2419 ilion ave n minneapolis mn, 2419 ilion ave n minneapolis mn), (2419 ilion ave n minneapolis mn, 2419 ilion avenue north minneapolis mn), (26 usc 132, 26 usc 132), (26 usc 132, irs section 132 f), (26 usc 132, tax code section 132), (2618 baltic rd blackville sc, 2618 baltic rd blackville sc), (2618 baltic rd blackville sc, 2618 baltic road, blackville, south carolina), (3% idaho, 3% idaho), (3% idaho, idaho 3%), (3311 eastport rd se dennison, 3311 eastport rd dennison ohio), (3311 eastport rd se dennison, 3311 eastport rd se dennison), (3311 eastport rd se dennison, 3311 eastport rd se dennison oh), (3311 eastport rd se dennison, 3311 eastport road dennison ohio), (4 e, 4 e), (4 e, 4 e's), (4 mile bike ride, 4 mile bike ride), (4 mile bike ride, how long to bike 4 miles), (40/50 percentage, 40/50 as a percentage), (40/50 percentage, 40/50 percentage), (4221 pc, 4221 pc), (4221 pc, compliance guide for 501(c)(3) public charities), (4221 pc, irs publication 4221), (4221 pc, irs publication 4221 pc), (4221 pc, irs publication 4221-pc), (5 bikes, 5 bikes), (5 bikes, bikes 5), (5 e, 5 e), (5 e, 5 e's), (5 e, 5 es), (5 e, 5.es), (5 e, the five e's), (5 miles on a bike, 5 miles on a bike), (5 miles on a bike, how long to bike 5 miles), (501c form pdf, 501 c 3 form pdf), (501c form pdf, 501c form pdf), (501c form pdf, sample 501c3 application pdf), (501c3 501c4, 501 c3 vs c4), (501c3 501c4, 501c3 501c4), (501c3 501c4, 501c3 or 501c4), (501c3 501c4, 501c3 vs 501c4), (501c3 501c4, 501c3 vs c4), (501c3 501c4, 501c4 vs 501c3), (501c3 501c4, difference between 501c3 and 501c4), (501c3 501c4, difference between 501c3 and c4), (501c3 501c4, difference between a 501c3 and 501c4), (501c3 501c4, what is a 501c4 vs 501c3), (501c3 501c4, what is the difference between 501c3 and 501c4), (501c3 501c4, what is the difference between a 501c3 and 501c4), (501c3 501c4, what is the difference between a 501c3 and a 501c4), (501c3 advocacy, 501c3 advocacy), (501c3 advocacy, 501c3 advocacy rules), (501c3 allowable expenses, 501c3 allowable expenses), (501c3 allowable expenses, tax write offs for nonprofit organizations), (501c3 application illinois, 501c3 application illinois), (501c3 application illinois, illinois 501 c 3 application), (501c3 benefits, 501c3 benefi

In [420]:
# save the clusters and queries
df_cluster.groupby(['serp_cluster','query']).first().reset_index().to_csv("clustered_queries_8000.csv")

In [424]:
# if using ahrefs then add back on the stats

df_cluster_results_wih_stats = df_cluster.merge(df_keywords_input, left_on="query", right_on="Keyword")
df_cluster_results_wih_stats.head()

,serp_cluster,query,Keyword,SERP features,Volume,KD,CPC,Previous traffic,Current traffic,Traffic change,Previous position,Current position,Position change,Previous URL inside,Previous URL,Current URL inside,Current URL,Previous date,Current date
0,(5,(5,(5,NaN,40,76,NaN,0.0,NaN,NaN,42.0,NaN,Lost,NaN,https://www.bikeleague.org/5-Es,NaN,NaN,2022-10-30 22:47:11,NaN
1,(5,@5,@5,NaN,50,76,NaN,0.0,NaN,NaN,69.0,NaN,Lost,NaN,https://www.bikeleague.org/5-Es,NaN,NaN,2022-10-31 00:23:47,NaN
2,(5,>5,>5,NaN,150,88,NaN,0.0,NaN,NaN,37.0,NaN,Lost,NaN,https://www.bikeleague.org/5-Es,NaN,NaN,2022-10-31 15:10:12,NaN
3,11 mile bike ride,biking 12 miles a day,biking 12 miles a day,NaN,40,1,NaN,0.0,NaN,NaN,95.0,NaN,Lost,NaN,https://www.bikeleague.org/content/bike-month-...,NaN,NaN,2022-10-29 08:54:32,NaN
4,11 mile bike ride,11 mile bike ride,11 mile bike ride,NaN,20,4,NaN,0.0,NaN,NaN,96.0,NaN,Lost,NaN,https://www.bikeleague.org/sites/default/files...,NaN,NaN,2022-10-28 14:01:16,NaN


In [449]:
df_cluster_with_stats = df_cluster_results_wih_stats.groupby('serp_cluster').agg({'Volume': ['mean', 'min', 'max', 'sum','count'], 'KD': ['mean', 'min', 'max'], 'CPC': ['mean', 'min', 'max']})

In [450]:
df_cluster_with_stats = df_cluster_with_stats.reset_index()

In [447]:
df_cluster_with_stats.columns

MultiIndex([('serp_cluster',     ''),
            (      'Volume', 'mean'),
            (      'Volume',  'min'),
            (      'Volume',  'max'),
            (      'Volume',  'sum'),
            (          'KD', 'mean'),
            (          'KD',  'min'),
            (          'KD',  'max'),
            (         'CPC', 'mean'),
            (         'CPC',  'min'),
            (         'CPC',  'max')],
           )

In [451]:
df_cluster_with_stats.sort_values( ("Volume","sum"), ascending=False )

serp_cluster        Volume                              \
                                           mean  min     max      sum count   
704                zzz_no_cluster    310.592386    0  343000  1892750  6094   
632                     theleague   5849.166667   30   69000    70190    12   
322                         fndly  16483.333333  200   49000    49450     3   
291                 donna edwards   5268.000000   20   26000    26340     5   
209                     busnesses   6090.000000   60   22000    24360     4   
..                            ...           ...  ...     ...      ...   ...   
481  neighborhood league wayne pa     10.000000   10      10       20     2   
440                map 21 funding     10.000000   10      10       20     2   
691               willard frances     20.000000   20      20       20     1   
208                  businesses's     20.000000   20      20       20     1   
110              bike classes nyc     10.000000   10      10       10     1   

            KD               CPC                
          mean min max      mean   min     max  
704  17.045290   0  99  1.423750  0.00  179.83  
632  67.000000  67  67  0.283333  0.00    1.27  
322  37.666667  36  39  0.435000  0.00    0.87  
291  18.600000   1  25  2.355000  0.33    4.38  
209  86.250000  75  92  1.220000  0.00    2.44  
..         ...  ..  ..       ...   ...     ...  
481   0.000000   0   0  0.060000  0.06    0.06  
440  25.000000  24  26  0.000000  0.00    0.00  
691  15.000000  15  15  0.000000  0.00    0.00  
208   2.000000   2   2       NaN   NaN     NaN  
110   4.000000   4   4       NaN   NaN     NaN  

[705 rows x 12 columns]

In [ ]:
df_cluster_results_wih_stats